In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/DI725/DI725-assignment2/

Mounted at /content/drive
/content/drive/MyDrive/DI725/DI725-assignment2


We will be working with the [AU-AIR dataset](https://bozcani.github.io/auairdataset), which consists of around 30k annotated low altitute traffic surveillance images with 8 object categories.

In [2]:
import os
import json
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import YolosForObjectDetection, YolosConfig, YolosImageProcessor
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback, WandbCallback
from transformers.trainer_callback import EarlyStoppingCallback
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import random
import wandb
import datetime
import pickle


# Configuration
DATA_DIR = "auair2019data"
IMAGES_DIR = os.path.join(DATA_DIR, "images")
ANNOTATIONS_FILE = os.path.join(DATA_DIR, "annotations.json")
OUTPUT_DIR = "yolos_finetuned"
MODEL_CHECKPOINT = "hustvl/yolos-small"
BATCH_SIZE = 4  # Smaller batch size to accommodate more images
LEARNING_RATE = 3e-5
NUM_EPOCHS = 20  # More epochs for large dataset
WARMUP_RATIO = 0.05  # Warmup ratio instead of fixed steps for large dataset
GRAD_ACCUMULATION_STEPS = 16  # for effective batch size of 64
NUM_WORKERS = 8  # For data loading
EVAL_STEPS = 200
SAVE_STEPS = 2000
EARLY_STOPPING_PATIENCE = 4  # Stop if no improvement for 4 evaluations

# Wandb configuration
WANDB_PROJECT = "object_detection_transformer"
WANDB_NAME = f"yolos-small-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"


# Seed everything for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

# Initialize wandb
print("Initializing wandb...")
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    config={
        "model_checkpoint": MODEL_CHECKPOINT,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "epochs": NUM_EPOCHS,
        "warmup_ratio": WARMUP_RATIO,
        "grad_accumulation_steps": GRAD_ACCUMULATION_STEPS,
        "effective_batch_size": BATCH_SIZE * GRAD_ACCUMULATION_STEPS,
        "early_stopping_patience": EARLY_STOPPING_PATIENCE,
    }
)

# Load annotations
print("Loading annotations...")
with open(ANNOTATIONS_FILE, 'r') as f:
    data = json.load(f)

# Extract category information
categories = {idx: cat for idx, cat in enumerate(data['categories'], start=0)}
id2label = {k: v for k, v in categories.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

print(f"Found {num_labels} categories: {id2label}")
wandb.config.update({"num_classes": num_labels, "classes": list(id2label.values())})

annotations = data["annotations"]

# class distribution
category_counts = {class_name: 0 for class_name in id2label.values()}
for image_data in annotations:
    if "bbox" in image_data:
        for bbox in image_data["bbox"]:
            class_id = bbox["class"]
            if class_id in id2label:
                category_counts[id2label[class_id]] += 1

wandb.log({"class_distribution": wandb.Table(
    columns=["Category", "Count"],
    data=[[cat, count] for cat, count in category_counts.items()]
)})



Initializing wandb...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yildizz-nisan (yildizz-nisan-middle-east-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loading annotations...
Found 8 categories: {0: 'Human', 1: 'Car', 2: 'Truck', 3: 'Van', 4: 'Motorbike', 5: 'Bicycle', 6: 'Bus', 7: 'Trailer'}


In [3]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset

class CustomObjectDetectionDataset(Dataset):
    def __init__(self, annotations, img_dir, processor, transform=None):
        self.annotations = annotations
        self.img_dir = img_dir
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Load annotation
        ann_data = self.annotations[idx]
        img_path = os.path.join(self.img_dir, ann_data["image_name"])

        # Load image safely
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (640, 640), color='gray')
            ann_data["bbox"] = []

        image_width, image_height = image.size

        # Prepare boxes & labels
        boxes = []
        labels = []

        for bbox in ann_data["bbox"]:
            x_min = bbox["left"]
            y_min = bbox["top"]
            width = bbox["width"]
            height = bbox["height"]

            x_max = x_min + width
            y_max = y_min + height

            # Normalize to [0, 1]
            x_min = max(0, x_min / image_width)
            y_min = max(0, y_min / image_height)
            x_max = min(1, x_max / image_width)
            y_max = min(1, y_max / image_height)

            # Only add valid boxes (where max > min) before any transformations
            if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(bbox["class"])

        boxes = np.array(boxes, dtype=np.float32) if boxes else np.zeros((0, 4), dtype=np.float32)
        labels = np.array(labels, dtype=np.int64) if labels else np.zeros((0,), dtype=np.int64)

        # Albumentations transform
        if self.transform and len(boxes) > 0:
            try:
                transformed = self.transform(image=np.array(image), bboxes=boxes.tolist(), labels=labels.tolist())
                image = transformed['image']
                boxes = transformed['bboxes']
                labels = transformed['labels']

                # Handle empty transform result
                if not boxes:
                    boxes = np.zeros((0, 4), dtype=np.float32)
                    labels = np.zeros((0,), dtype=np.int64)
                else:
                    boxes = np.array(boxes, dtype=np.float32)
                    labels = np.array(labels, dtype=np.int64)

                    # Clip out-of-bounds
                    boxes = np.clip(boxes, 0.0, 1.0)

                    # Filter out invalid boxes
                    valid_indices = []
                    for i, box in enumerate(boxes):
                        x_min, y_min, x_max, y_max = box
                        if x_max > x_min and y_max > y_min:
                            valid_indices.append(i)

                    boxes = boxes[valid_indices] if valid_indices else np.zeros((0, 4), dtype=np.float32)
                    labels = labels[valid_indices] if valid_indices else np.zeros((0,), dtype=np.int64)

            except Exception as e:
                print(f"Error during transformation: {e}")
                # Fall back to original image and empty boxes/labels
                image = np.array(image)
                boxes = np.zeros((0, 4), dtype=np.float32)
                labels = np.zeros((0,), dtype=np.int64)
        elif self.transform:
            # If there are no boxes but we have a transform
            image = self.transform(image=np.array(image))['image']

        # Construct annotations for processor
        annotations = []
        for i, (box, label) in enumerate(zip(boxes, labels)):
            x_min, y_min, x_max, y_max = box
            # Double-check validity of box
            if x_max <= x_min or y_max <= y_min:
                continue

            area = (x_max - x_min) * (y_max - y_min)
            annotations.append({
                'bbox': [x_min, y_min, x_max, y_max],
                'category_id': int(label),
                'area': float(area),
                'iscrowd': 0
            })

        # Feed image + annotations into YOLOS processor
        encoding = self.processor(
            images=image,
            annotations={
                'image_id': idx,
                'annotations': annotations
            },
            return_tensors="pt"
        )

        # Remove batch dimension
        for k, v in encoding.items():
            if isinstance(v, torch.Tensor):
                encoding[k] = v.squeeze(0)  # remove batch dim only
            else:
                encoding[k] = v  # leave non-tensor items as-is

        return encoding

In [4]:

# Initialize image processor
print(f"Loading YOLOS image processor from {MODEL_CHECKPOINT}")
processor = YolosImageProcessor.from_pretrained(MODEL_CHECKPOINT)

# Define data augmentations

train_transform = A.Compose(
    [
        A.Resize(640, 640),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.ShiftScaleRotate(
            shift_limit=0.05, scale_limit=0.1, rotate_limit=10, border_mode=0, p=0.5
        ),
    ],
    bbox_params=A.BboxParams(
        format='yolo',              #
        label_fields=['labels'],
        min_visibility=0.1,         # Discard boxes too small or barely visible
        clip=True,            # clips bboxes to [0, 1]
    )
)

val_transform = A.Compose(
    [
        A.Resize(640, 640)  #
    ],
    bbox_params=A.BboxParams(
        format='yolo',
        label_fields=['labels'],
        clip=True,
    )
)


# Log augmentation pipeline to wandb
wandb.config.update({
    "augmentations": str(train_transform),
})

# Create dataset
print("Creating datasets...")
full_dataset = CustomObjectDetectionDataset(
    annotations,
    IMAGES_DIR,
    processor
)

# If we haven't already split the dataset:
if not os.path.exists(f"{OUTPUT_DIR}/dataset_splits.pkl"):

    # subsetting the data because it is too big
    subset_size = int(len(full_dataset) * 0.3)

    # Create random subset indices
    indices = torch.randperm(len(full_dataset))[:subset_size]

    # Create subset
    subset_dataset = torch.utils.data.Subset(full_dataset, indices)

    # Split dataset
    train_size = int(0.7 * len(subset_dataset))
    remaining = len(subset_dataset) - train_size
    val_size = remaining // 2      # 15% of subset
    test_size = remaining - val_size  # 15% of subset
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(subset_dataset, [train_size, val_size, test_size])

    # Save splits using indices for reproducibility
    split_indices = {
        "subset_indices": indices,
        "train_indices": train_dataset.indices,
        "val_indices": val_dataset.indices,
        "test_indices": test_dataset.indices
    }

    with open(f"{OUTPUT_DIR}/dataset_splits.pkl", "wb") as f:
        pickle.dump(split_indices, f)

with open(f"{OUTPUT_DIR}/dataset_splits.pkl", "rb") as f:
    loaded_splits = pickle.load(f)

#subset the data
subset_dataset = torch.utils.data.Subset(full_dataset, loaded_splits["subset_indices"])

# Recreate datasets using original dataset
train_dataset = torch.utils.data.Subset(subset_dataset, loaded_splits["train_indices"])
val_dataset = torch.utils.data.Subset(subset_dataset, loaded_splits["val_indices"])
test_dataset = torch.utils.data.Subset(subset_dataset, loaded_splits["test_indices"])


# Apply transforms
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform = val_transform

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


# Log data split info
wandb.config.update({
    "train_size": len(train_dataset),
    "val_size": len(val_dataset),
    "train_val_ratio": train_size / val_size if val_size > 0 else "N/A",
})


Loading YOLOS image processor from hustvl/yolos-small


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

Creating datasets...
Train dataset size: 6892
Validation dataset size: 1477
Test dataset size: 1477


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [6]:
from transformers import TrainerCallback

def collate_fn(batch):
    batch_dict = {
        "pixel_values": torch.stack([item["pixel_values"] for item in batch])
    }

    # For YOLOS, we need to keep labels as a list of label dictionaries
    if "labels" in batch[0]:
        batch_dict["labels"] = [item["labels"][0] for item in batch]

    return batch_dict

# Get model with updated config for your number of classes
print(f"Loading YOLOS model from {MODEL_CHECKPOINT}")
config = YolosConfig.from_pretrained(MODEL_CHECKPOINT, id2label=id2label, label2id=label2id)
model = YolosForObjectDetection.from_pretrained(MODEL_CHECKPOINT, config=config, ignore_mismatched_sizes=True)

# Log model architecture summary to wandb
wandb.config.update({
    "model_params": sum(p.numel() for p in model.parameters()),
    "trainable_params": sum(p.numel() for p in model.parameters() if p.requires_grad),
})

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Create a custom callback to log losses to wandb and print progress
class WandbLoggingCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None:
            # Calculate epoch from global step
            epoch = state.epoch

            # Print evaluation information
            print(f"\n=== Evaluation at Step {state.global_step} (Epoch {epoch:.2f}) ===")
            for key, value in metrics.items():
                if key != "epoch":
                    print(f"Eval {key}: {value:.5f}")

            # Log eval metrics to wandb
            wandb.log(
                {f"eval/{k}": v for k, v in metrics.items() if k != "epoch"},
                step=state.global_step
            )

    def on_log(self, args, state, control, logs=None, **kwargs):
        # Log training metrics
        if logs is not None:
            # Print training information
            if "loss" in logs:
                print(f"Step {state.global_step} (Epoch {logs.get('epoch', state.epoch):.2f}) - Training loss: {logs['loss']:.5f}")

            # Filter out eval metrics which are handled in on_evaluate
            train_logs = {k: v for k, v in logs.items() if not k.startswith("eval_")}
            wandb.log(
                {f"train/{k}": v for k, v in train_logs.items() if k != "epoch"},
                step=state.global_step
            )

# Define training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=False,
    fp16=True,
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    save_total_limit=3,
    dataloader_num_workers=NUM_WORKERS,
    dataloader_drop_last=True,
    dataloader_pin_memory=True,
    report_to="wandb",
    gradient_checkpointing=True,
    ddp_find_unused_parameters=False,
    logging_strategy="steps",  # Log at the same frequency as evaluation
    logging_steps=EVAL_STEPS // 5,  # Log more frequently than evaluation for better tracking
    logging_first_step=True,  # Log the first step to get initial loss
)

# Initialize Trainer with early stopping and wandb logging
print("Initializing the Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=EARLY_STOPPING_PATIENCE),
        WandbLoggingCallback(),  # Add custom wandb logging callback
    ],
)

# Train the model
print("\n" + "="*50)
print("Starting training...")
print(f"Number of epochs: {NUM_EPOCHS}")
print(f"Training batch size: {BATCH_SIZE}")
print(f"Evaluation steps: Every {EVAL_STEPS} steps")
print(f"Saving steps: Every {SAVE_STEPS} steps")
print(f"Early stopping patience: {EARLY_STOPPING_PATIENCE} evaluations")
print("="*50 + "\n")

train_result = trainer.train()

# Print training summary
print("\n" + "="*50)
print("Training completed!")
print(f"Total steps: {trainer.state.global_step}")
print(f"Final training loss: {train_result.training_loss:.5f}")
print("="*50 + "\n")

# Save the final model
final_model_path = os.path.join(OUTPUT_DIR, "final_model")
trainer.save_model(final_model_path)
print(f"Model saved to {final_model_path}")

# Create a validation dataset subset for final evaluation
print("\nPerforming final evaluation...")
eval_subset_size = min(1000, len(val_dataset))
eval_subset_indices = torch.randperm(len(val_dataset))[:eval_subset_size]
eval_subset = torch.utils.data.Subset(val_dataset, eval_subset_indices)

# Evaluate the model on the validation subset
eval_results = trainer.evaluate(eval_dataset=eval_subset)
print("\n" + "="*50)
print("Final evaluation results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.5f}")
print("="*50 + "\n")

# Finish wandb run
print("Finishing wandb run...")
wandb.finish()
print("Training complete!")

Loading YOLOS model from hustvl/yolos-small


Some weights of YolosForObjectDetection were not initialized from the model checkpoint at hustvl/yolos-small and are newly initialized because the shapes did not match:
- class_labels_classifier.layers.2.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([9]) in the model instantiated
- class_labels_classifier.layers.2.weight: found shape torch.Size([92, 384]) in the checkpoint and torch.Size([9, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initializing the Trainer...

Starting training...
Number of epochs: 20
Training batch size: 4
Evaluation steps: Every 200 steps
Saving steps: Every 2000 steps
Early stopping patience: 4 evaluations



The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Step,Training Loss,Validation Loss
200,1.627200,1.695755
400,1.450900,1.467428
600,1.375500,1.396618
800,1.339400,1.411289
1000,1.310100,1.359288
1200,1.276200,1.326345
1400,1.230000,1.264164
1600,1.192900,1.233741
1800,1.188100,1.215379
2000,1.125500,1.203504


...

Step 2080 (Epoch 19.26) - Training loss: 1.12140
Step 2120 (Epoch 19.63) - Training loss: 1.12360

Training completed!
Total steps: 2140
Final training loss: 1.45783

Model saved to yolos_finetuned/final_model

Performing final evaluation...


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.



=== Evaluation at Step 2140 (Epoch 19.82) ===
Eval eval_loss: 1.21375
Eval eval_runtime: 21.00050
Eval eval_samples_per_second: 47.61800
Eval eval_steps_per_second: 11.90400

Final evaluation results:
eval_loss: 1.21375
eval_runtime: 21.00050
eval_samples_per_second: 47.61800
eval_steps_per_second: 11.90400
epoch: 19.81718

Finishing wandb run...


eval/loss,█▅▄▄▃▃▂▁▁▁▁
eval/runtime,█▁▁▁▁▁▁▁▁▁▁
eval/samples_per_second,▁██████████
eval/steps_per_second,▁██████████
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇██
train/grad_norm,▄▄▅▄▃▄▄▂▄▄▄▄▅▅▄▄▆▂▂▃█▄▃▂▃▂▂▂▄▄▅▃▅▃▃▃▁▃▃▅
train/learning_rate,▁▃▆███▇▇▇▇▇▇▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁
train/loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


Training complete!
